In [1]:
import pandas as pd 
import numpy as np 

In [2]:
data_path = '../data_sources/psid/'

In [3]:
df = pd.read_stata(data_path+'pequiv_1989.dta',convert_categoricals=False)

In [4]:
df.columns = [c.replace('_1989','') for c in df.columns]
df['year'] = 1989

In [5]:
for t in range(1990,1998):
	dft = pd.read_stata(data_path+'pequiv_'+str(t)+'.dta',convert_categoricals=False)
	dft.columns = [c.replace('_'+str(t),'') for c in dft.columns]
	dft['year'] = t
	df = df.append(dft)

In [6]:
df['year'].value_counts().sort_index()

1989    19669
1990    19932
1991    19962
1992    20334
1993    21450
1994    23620
1995    23182
1996    23060
1997    19132
Name: year, dtype: int64

In [7]:
df = df[df['d11105']==1]

In [8]:
df = df.rename({'x11101ll':'pid'},axis=1)
df = df.rename({'d11101':'age'},axis=1)
df = df.rename({'i11113':'Y'},axis=1)
df['co'] = 'US'
df = df[['co','pid','year','age','Y']]
df = df[df['Y']>0]
p99 = df['Y'].quantile(0.99)
df = df[df['Y']<p99]
df['Y'] = df['Y']/df['Y'].mean()
df['logY'] = np.where((df['Y']>0) & (df['Y'].isna()==False),np.log(df['Y']),np.nan)

In [9]:
df.head()

,co,pid,year,age,Y,logY
0,US,4001,1989,67.0,0.763534,-0.269798
2,US,4003,1989,38.0,0.954480,-0.046588
6,US,4008,1989,24.0,0.725096,-0.321451
15,US,4170,1989,34.0,0.836848,-0.178113
16,US,4172,1989,40.0,1.657952,0.505583


In [10]:
df.groupby('year').mean()['Y']

year
1989    0.887920
1990    0.914923
1991    0.946179
1992    0.970310
1993    1.015430
1994    0.979302
1995    1.023467
1996    1.080029
1997    1.190752
Name: Y, dtype: float64

In [11]:
df['wave'] = df['year'] - 1989 + 1

In [12]:
df.to_pickle(data_path+'psid_incomes.pkl')